# Seleção das estações e poluentes
As estações tem medições horárias.

Para que um dia seja considerado válido **para uma estação e um poluente** é necessário que tenha pelo menos 75% das obervações, ou seja pelo menos 18 observações.

Para que um ano seja considerado válido **para uma estação e um poluente** é necessário que tenha pelo menos 75% dos dias válidos ou seja pelo menos 328 dias.

Para que um poluente de uma estação seja considerado para as análises posteriores tem que ter pelo menos 2 anos válidos

## Inicialização

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Diretoria onde os dados estão localizados
dir_files = './03-dados-qualar-longo-corrigido/'

In [3]:
# Ficheiro das medições 
ficheiro_medicoes = dir_files + '03-medicoes-longo.csv'

In [4]:
medicoes = pd.read_csv(ficheiro_medicoes, thousands=',', index_col=0, parse_dates=True)
medicoes

,Estação,variable,value
data,,,
2001-11-08 00:00:00,Afonso III,SO2,0.0
2001-11-08 01:00:00,Afonso III,SO2,0.0
2001-11-08 02:00:00,Afonso III,SO2,0.0
2001-11-08 03:00:00,Afonso III,SO2,0.0
2001-11-08 04:00:00,Afonso III,SO2,0.0
...,...,...,...
2009-12-31 19:00:00,Vermoim,PM2.5,2.0
2009-12-31 20:00:00,Vermoim,PM2.5,4.0
2009-12-31 21:00:00,Vermoim,PM2.5,5.0


In [5]:
# Criar uma coluna com apenas o dia da data
medicoes['Dia'] =medicoes.index.date
# Criar uma coluna com o ano da data
medicoes['Ano'] = medicoes.index.year
# Conferir resultados
medicoes

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


,Estação,variable,value,Dia,Ano
data,,,,,
2001-11-08 00:00:00,Afonso III,SO2,0.0,2001-11-08,2001
2001-11-08 01:00:00,Afonso III,SO2,0.0,2001-11-08,2001
2001-11-08 02:00:00,Afonso III,SO2,0.0,2001-11-08,2001
2001-11-08 03:00:00,Afonso III,SO2,0.0,2001-11-08,2001
2001-11-08 04:00:00,Afonso III,SO2,0.0,2001-11-08,2001
...,...,...,...,...,...
2009-12-31 19:00:00,Vermoim,PM2.5,2.0,2009-12-31,2009
2009-12-31 20:00:00,Vermoim,PM2.5,4.0,2009-12-31,2009
2009-12-31 21:00:00,Vermoim,PM2.5,5.0,2009-12-31,2009


## Cálculo de horas por dia e dias válidos
Vamos analisar a quantidade de dias válidos presentes nos dados.

In [6]:
lista_estacoes = medicoes['Estação'].unique()
lista_estacoes

array(['Afonso III', 'Alfragide/Amadora', 'Alto Seixalinho', 'Alverca',
       'Anta-Espinho', 'Avintes', 'Beato', 'Benfica', 'Boavista',
       'Burgães-Santo Tirso', 'Camarinha', 'Centro de Lacticínios',
       'Cerro', 'Chamusca', 'Chelas', 'Coimbra/ Avenida Fernão Magalhães',
       'Custóias-Matosinhos', 'D.Manuel II-Vermoim', 'Douro Norte',
       'Entrecampos', 'Ermesinde-Valongo', 'Ervedeira', 'Escavadeira',
       'Espinho', 'Estarreja', 'Estarreja/Avanca', 'Fernando Pó',
       'Fidalguinhos', 'Formosa', 'Fornelo do Monte', 'Frossos-Braga',
       'Fundão', 'Horto', 'Instituto Geofísico de Coimbra',
       'Joaquim Magalhães', 'João Gomes Laranjo-S.Hora', 'Laranjeiro',
       'Lavradio', 'Leça do Balio-Matosinhos', 'Loures-Centro',
       'Lourinhã', 'Malpique', 'Matosinhos', 'Meco-Perafita',
       'Mem Martins', 'Mindelo-Vila do Conde', 'Minho-Lima',
       'Montemor-o-Velho', 'Monte Chãos', 'Monte Velho', 'Olivais',
       'Paio Pires', 'Paços de Ferreira', 'Quebedo', 'Qui

In [7]:
lista_poluentes = medicoes['variable'].unique()
lista_poluentes

array(['SO2', 'PM10', 'O3', 'NO2', 'PM2.5'], dtype=object)

In [8]:
# Cacular horas por dia
validade_diaria = medicoes.groupby(['Estação', 'variable', 'Dia']).size().reset_index(name = 'Horas')

In [9]:
validade_diaria

,Estação,variable,Dia,Horas
0,Afonso III,NO2,2001-11-08,11
1,Afonso III,NO2,2001-11-23,24
2,Afonso III,NO2,2001-11-24,24
3,Afonso III,NO2,2001-11-25,24
4,Afonso III,NO2,2001-11-26,24
...,...,...,...,...
1371991,Ílhavo,SO2,2022-12-27,24
1371992,Ílhavo,SO2,2022-12-28,24
1371993,Ílhavo,SO2,2022-12-29,24
1371994,Ílhavo,SO2,2022-12-30,24


In [10]:
# Calcular dias válidos
validade_diaria['Validade'] = validade_diaria['Horas'].apply(lambda x: 'Válido' if x>=18 else 'Inválido')
validade_diaria

,Estação,variable,Dia,Horas,Validade
0,Afonso III,NO2,2001-11-08,11,Inválido
1,Afonso III,NO2,2001-11-23,24,Válido
2,Afonso III,NO2,2001-11-24,24,Válido
3,Afonso III,NO2,2001-11-25,24,Válido
4,Afonso III,NO2,2001-11-26,24,Válido
...,...,...,...,...,...
1371991,Ílhavo,SO2,2022-12-27,24,Válido
1371992,Ílhavo,SO2,2022-12-28,24,Válido
1371993,Ílhavo,SO2,2022-12-29,24,Válido
1371994,Ílhavo,SO2,2022-12-30,24,Válido


In [11]:
# Mostrar número de dias válidos/não válidos nos dados
dias_validos = validade_diaria['Validade'].value_counts()
dias_validos

Validade
Válido      1327813
Inválido      44183
Name: count, dtype: int64

## Cálculo dos dias válidos por ano e anos válidos
Vamos analisar a quantidade de anos válidos presentes nos dados.

In [12]:
# Preparar dataframe com informação de dias válidos e ano
dias_validados = validade_diaria.loc[validade_diaria['Validade'] == 'Válido']
dias_validados = dias_validados.copy()
dias_validados['Ano'] = dias_validados['Dia'].apply(lambda x: x.year)
dias_validados

,Estação,variable,Dia,Horas,Validade,Ano
1,Afonso III,NO2,2001-11-23,24,Válido,2001
2,Afonso III,NO2,2001-11-24,24,Válido,2001
3,Afonso III,NO2,2001-11-25,24,Válido,2001
4,Afonso III,NO2,2001-11-26,24,Válido,2001
5,Afonso III,NO2,2001-11-27,24,Válido,2001
...,...,...,...,...,...,...
1371991,Ílhavo,SO2,2022-12-27,24,Válido,2022
1371992,Ílhavo,SO2,2022-12-28,24,Válido,2022
1371993,Ílhavo,SO2,2022-12-29,24,Válido,2022
1371994,Ílhavo,SO2,2022-12-30,24,Válido,2022


In [13]:
# Calcular dias válidos por ano
validade_anual = dias_validados.groupby(['Estação', 'variable', 'Ano']).size().reset_index(name = 'Dias')
validade_anual

,Estação,variable,Ano,Dias
0,Afonso III,NO2,2001,35
1,Afonso III,NO2,2002,357
2,Afonso III,NO2,2003,331
3,Afonso III,NO2,2004,344
4,Afonso III,NO2,2005,300
...,...,...,...,...
4347,Ílhavo,SO2,2018,351
4348,Ílhavo,SO2,2019,364
4349,Ílhavo,SO2,2020,330
4350,Ílhavo,SO2,2021,281


In [14]:
# Calcular anos válidos por estação e poluente
validade_anual['Validade'] = validade_anual['Dias'].apply(lambda x: 'Válido' if x>=328 else 'Inválido')
validade_anual

,Estação,variable,Ano,Dias,Validade
0,Afonso III,NO2,2001,35,Inválido
1,Afonso III,NO2,2002,357,Válido
2,Afonso III,NO2,2003,331,Válido
3,Afonso III,NO2,2004,344,Válido
4,Afonso III,NO2,2005,300,Inválido
...,...,...,...,...,...
4347,Ílhavo,SO2,2018,351,Válido
4348,Ílhavo,SO2,2019,364,Válido
4349,Ílhavo,SO2,2020,330,Válido
4350,Ílhavo,SO2,2021,281,Inválido


In [15]:
# Mostrar número de anos válidos/não válidos nos dados
anos_validos = validade_anual['Validade'].value_counts()
anos_validos

Validade
Válido      2707
Inválido    1645
Name: count, dtype: int64

## Medições de poluentes e estações válidas
Vamos manter apenas as medições correspondentes a anos válidos.

In [16]:
# Juntar informação de ano válido às medições
anos_validados = medicoes.merge(validade_anual[['Estação', 'variable', 'Ano', 'Validade']], on=['Estação', 'variable', 'Ano'], how='inner')
anos_validados

,Estação,variable,value,Dia,Ano,Validade
0,Afonso III,SO2,0.0,2001-11-08,2001,Inválido
1,Afonso III,SO2,0.0,2001-11-08,2001,Inválido
2,Afonso III,SO2,0.0,2001-11-08,2001,Inválido
3,Afonso III,SO2,0.0,2001-11-08,2001,Inválido
4,Afonso III,SO2,0.0,2001-11-08,2001,Inválido
...,...,...,...,...,...,...
32086023,Vermoim,PM2.5,2.0,2009-12-31,2009,Válido
32086024,Vermoim,PM2.5,4.0,2009-12-31,2009,Válido
32086025,Vermoim,PM2.5,5.0,2009-12-31,2009,Válido
32086026,Vermoim,PM2.5,2.0,2009-12-31,2009,Válido


In [17]:
# Manter apenas as medições de anos válidos
anos_validados = anos_validados.loc[anos_validados['Validade'] == 'Válido']
anos_validados

,Estação,variable,value,Dia,Ano,Validade
26829,Alfragide/Amadora,SO2,61.2,2002-01-01,2002,Válido
26830,Alfragide/Amadora,SO2,39.9,2002-01-01,2002,Válido
26831,Alfragide/Amadora,SO2,26.6,2002-01-01,2002,Válido
26832,Alfragide/Amadora,SO2,21.3,2002-01-01,2002,Válido
26833,Alfragide/Amadora,SO2,13.3,2002-01-01,2002,Válido
...,...,...,...,...,...,...
32086023,Vermoim,PM2.5,2.0,2009-12-31,2009,Válido
32086024,Vermoim,PM2.5,4.0,2009-12-31,2009,Válido
32086025,Vermoim,PM2.5,5.0,2009-12-31,2009,Válido
32086026,Vermoim,PM2.5,2.0,2009-12-31,2009,Válido


In [18]:
# Calcular medições válidas por estação
med_validas_estacao = anos_validados.groupby(['Estação']).size()
med_validas_estacao

Estação
Afonso III            152437
Alfragide/Amadora     316209
Alto Seixalinho        76134
Alverca               381423
Anta-Espinho           93612
                       ...  
Vermoim               299119
Vila Nova da Telha     34516
Vila do Conde         144923
Águas Santas           59459
Ílhavo                439860
Length: 90, dtype: int64

In [19]:
# Calcular medições válidas por poluente
med_validas_poluente = anos_validados.groupby(['variable']).size()
med_validas_poluente

variable
NO2      6374778
O3       5615690
PM10     5452260
PM2.5    1376811
SO2      4136334
dtype: int64